<a href="https://colab.research.google.com/github/roger21gm/neural-network-impl/blob/master/NeuralNetworkImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>